# Basic OFDM Example in Python
In this notebook, we will investigate the basic building blocks of an OFDM system at the transmitter and receiver side. OFDM (Orthogonal frequency division multiplexing) is a multicarrier system that is applied in a wide range of wireless transmission systems, such as LTE, WiMAX and DVB-T and DAB. The fundamental concept of a multicarrier system is the division of a high-rate transmitted data stream into several low-rate narrow subcarriers. This way, several advantages are obtained:

Since the symbol duration is inverse proportional to the symbol rate, each subcarrier has relatively long symbols. Long symbols are robust against multipath fading, as it occurs in wireless systems.
When a carrier is in a deep fade due to frequency-selectivity of the channel (i.e. the received energy on this carrier is very low), only the data on this subcarrier is lost, instead of the whole stream.
Multicarrier systems allow easy multi-user resource sharing by allocating different subcarriers to different users.
Consider the following block diagram, which contains fundamental blocks for the OFDM system:

In the following OFDM example, we will go through each block and describe its operation. However, before let us define some parameters that are used for the OFDM system:

The number of subcarriers $K$ describes, how many subcarriers are available in the OFDM system.


In [ ]:
pkg load communications
K = 64 % number of OFDM subcarriers

The length of the cyclic prefix (CP) denotes the number of samples that are copied from the end of the modulated block to the beginning, to yield a cyclic extension of the block. There is a dedicated article on the CP of OFDM which treats its application in more detail.

In [ ]:
CP = K/4  % length of the cyclic prefix: 25% of the block

The number of pilots ***P*** in the OFDM symbol describes, how many carriers are used to transmit known information (i.e. pilots). Pilots will be used at the receiver to estimate the wireless channel between transmitter and receiver. Further, we also define the value that each pilots transmits (which is known to the receiver).

In [ ]:
P = 8 % number of pilot carriers per OFDM block
pilotValue = 3 + 3i % The known value each pilot transmits

Now, let us define some index sets that describe which carriers transmit pilots and which carriers contain payload.



In [ ]:
allCarriers = 1:1:K;  # indices of all subcarriers ([0, 1, ... K-1])

pilotCarriers = allCarriers(1:7:end); # Pilots is every (K/P)th carrier.

# For convenience of channel estimation, let's make the last carriers also be a pilot
pilotCarriers = [pilotCarriers allCarriers(end)]
P = P+1

# data carriers are all remaining carriers
idx               = ismember(allCarriers,pilotCarriers);
dataCarriers      = allCarriers;
dataCarriers(idx) = []

plot(pilotCarriers, zeros(1,length(pilotCarriers)), 'bo'); hold on; % label='pilot'
plot(dataCarriers,  zeros(1,length(dataCarriers)), 'ro'); hold off; %  label='data'

Let's define the modulation index $μ$ and the corresponding mapping table. We consider 16QAM transmission, i.e. we have $μ=4$ bits per symbol. Furthermore, the mapping from groups of 4 bits to a 16QAM constellation symbol shall be defined in ```mapping_table```.

In [ ]:
mu = 4 % bits per symbol (i.e. 16QAM)
payloadBits_per_OFDM = length(dataCarriers)*mu  # number of payload bits per OFDM symbol

mapping_table = [
    -3-3j, -3-1j, -3+3j, -3+1j,
    -1-3j, -1-1j, -1+3j, -1+1j,
    3-3j,  3-1j,  3+3j,  3+1j,
    1-3j,  1-1j,  1+3j,  1+1j 
    ]
for b3 = 0:1
    for b2 = 0:1
        for b1 = 0:1
            for b0 = 0:1
                B = con b3 b2, b1, b0
                Q = mapping_table[B]
                plot(real(Q), imag(Q), 'bo')
                plot(real(Q), imag(Q)+0.2, num2str(x) for x in B), ha='center')

In [ ]:
imag(pilotValue)

In [ ]:
num2str(110)